# $\beta$ -Pic b planet

I'll try to obtain the same posteriors as found in the paper of [Sun et al](https://arxiv.org/pdf/2201.08506.pdf)

## Import the necessary packages

In [6]:
from prior import prior_distributions
from simulator import OrbitCreator
from training import train_sbi

import torch

import orbitize
from orbitize import read_input

from lampe.data import H5Dataset
from lampe.inference import * # Import all methods and their losses

## Load the observations

In [4]:
data_set = read_input.read_file('{}/betaPic.csv'.format(orbitize.DATADIR))

data_set = data_set[:-1] # Discard the  RV observation, don't know how to take it into account

## Define the priors
The priors are defined as such 
$$
\begin{align*}
a &\sim \log \mathcal{U}(10,10^{4})\\
e &\sim \mathcal{U}(10^{8}, 0.99)\\
i &\sim \mathcal{U}(0,180)\\
\omega &\sim \mathcal{U}(0,360)\\
\Omega &\sim \mathcal{U}(0,360)\\
\tau &\sim \mathcal{U}(0,1)\\
\pi &\sim \mathcal{N}(56.95, 0.26) \\
M_T &\sim \mathcal{N}(1.22,0.08)\\
\end{align*}
$$


In [9]:
prior = prior_distributions(log_uniform_lower = torch.tensor(10.0), 
                            log_uniform_upper = torch.tensor(10**4),
                            uniform_lower = torch.tensor([10e-8, 0.0, 0.0, 0.0, 0.0]), 
                            uniform_upper = torch.tensor([0.99, 180.0, 360.0, 360.0, 1.0]),
                            gaussian_mean = torch.tensor([56.95, 1.22]), 
                            gaussian_std = torch.tensor([0.26, 0.08]))

## Instantiate the simulator

And test if it works well

In [31]:
simulator = OrbitCreator(data_set)

thetas = prior.sample((1,))
x = simulator(thetas)

label_print = ['a', 'e', 'i', 'ω', 'Ω', 'τ', 'π', 'Mt']

for label, theta_value in zip(label_print, thetas[0].tolist()):
    print(f"{label}: {theta_value:.3f}")

print("\n x:")
for i in range(0, len(x[0]), 2):
    x_coord = round(x[0][i].item(), 3)
    y_coord = round(x[0][i+1].item(), 3)
    print(f"({x_coord}, {y_coord})", end="\n")

a: 19.097
e: 0.872
i: 143.785
ω: 296.819
Ω: 114.182
τ: 0.267
π: 57.408
Mt: 1.199

 x:
(-33.042, 1036.183)
(-213.696, 1166.674)
(2.233, 994.635)
(8.531, 959.258)
(18.718, 957.308)
(36.111, 928.944)
(41.008, 929.354)
(41.461, 910.319)
(48.353, 915.522)
(54.841, 909.767)
(6.623, 977.997)
(5.396, 992.322)
(44.872, 920.73)
(74.746, 875.966)
(74.813, 871.634)
(90.791, 848.704)
(90.846, 844.541)
(115.632, 796.886)
(115.041, 801.431)
(148.611, 741.013)
(148.608, 743.667)
(148.797, 746.189)
(150.863, 743.167)
(150.865, 740.291)
(150.955, 740.755)
(181.201, 670.708)
(193.733, 637.076)
(211.498, 582.907)
(213.727, 579.116)
(215.026, 572.954)
(217.306, 565.454)
(187.922, 653.284)
(248.554, 234.085)
(244.742, 206.38)


## Load the generated dataset

In [ ]:
trainset = H5Dataset('Datasets/beta-pic-train.h5',
                     batch_size=2048, shuffle=True)
validset = H5Dataset('Datasets/beta-pic-val.h5',
                     batch_size=2048)
testset = H5Dataset('Datasets/beta-pic-test.h5')

## Train the SBI model

In [ ]:
len_obs = len(data_set)

estimator = NRE(8, len_obs, hidden_features=[256]*5).cuda()

train_sbi('NRE',
          estimator, 
          NRELoss, 
          trainset, 
          validset, 
          epochs = 128)